# Import

In [1]:
import pandas as pd
import numpy as np
import gc
import os
from sklearn.preprocessing import LabelEncoder

# to parquet

In [4]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [8]:
csv_to_parquet('./train.csv', 'train')
csv_to_parquet('./test.csv', 'test')

train Done.
test Done.


# Data load

In [2]:
from email.utils import parsedate


train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [3]:
train.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


# Preprocess

In [4]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [5]:
y_train = train['target'] 

X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 17)
(4701217,)
(291241, 17)


In [6]:
from tqdm import tqdm

def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length])

    return np.array(result)

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# train
# train, validation 분리 (8 : 2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=119)

# scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# time series window 생성
X_train = time_window(X_train_scaled, 0, 1)
X_val = time_window(X_val_scaled, 0, 1)
y_train = time_window(y_train, 0, 1)
y_val = time_window(y_val, 0, 1)

# y의 길이와 같은 길이로 설정
X_train = X_train[:len(y_train)]
X_val = X_val[:len(y_val)]

100%|██████████| 940243/940243 [00:22<00:00, 42125.58it/s]


# Modeling

## transformer 정의

### encoder

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
import random

# 시드고정
tf.random.set_seed(42)
random.seed(42)
np.random.seed(42)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):

    x = layers.LayerNormalization(epsilon=epsilon)(inputs) # 레이어 정규화
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=epsilon)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation=activation)(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

### build

In [9]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation=activation)(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

## earlystop patience

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=earlystop_patience)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

## train

In [11]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=head_size,
    num_heads=num_heads,
    ff_dim=ff_dim,
    num_transformer_blocks=num_transformer_blocks,
    mlp_units=mlp_units,
    mlp_dropout=mlp_dropout,
    dropout=dropout,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

## set parameter

In [12]:
# parameter

epsilon = 1e-6
epoch = 100 # 400~500
batch = 2048 # 10
learning_rate = 0.03 # lr 늘려도 될 것 같음
head_size = 256
num_heads = 4
ff_dim = 4
num_transformer_blocks = 4
mlp_units = [64]
mlp_dropout = 0.1
dropout = 0.1
activation = 'swish'
earlystop_patience = 10

# Predict

In [13]:
from tensorflow import keras


# transformer 모델 훈련
transformer = train(X_train, y_train, X_val, y_val, f'transformer-{i}', epoch,
                    batch, learning_rate)
transformer.load_weights(f'./check/transformer-{i}-{epoch}-{batch}.h5')

if os.path.exists(f'./model') == False:
    os.mkdir(f'./model')

# 모델 저장
transformer.save(f'./model/transformer-{i}-{epoch}-{batch}.h5')


# test
# scaling
test_scaled = scaler.transform(test)

# reshape
test = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

# predict
model_test = tf.keras.models.load_model(f'./model/transformer-{i}-{epoch}-{batch}.h5')
pred = model_test.predict(test)

# 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit.csv", index = False)

sample_submission

Epoch 1/100
1836/1836 [==============================] - 100s 49ms/step - loss: 134.6431 - val_loss: 88.1462

Epoch 00001: val_loss improved from inf to 88.14616, saving model to ./check\transformer-end_turn_restricted-100-2048.h5
Epoch 2/100
1836/1836 [==============================] - 88s 48ms/step - loss: 100.8083 - val_loss: 84.1279

Epoch 00002: val_loss improved from 88.14616 to 84.12787, saving model to ./check\transformer-end_turn_restricted-100-2048.h5
Epoch 3/100
1836/1836 [==============================] - 88s 48ms/step - loss: 97.0826 - val_loss: 81.5202

Epoch 00003: val_loss improved from 84.12787 to 81.52022, saving model to ./check\transformer-end_turn_restricted-100-2048.h5
Epoch 4/100
1836/1836 [==============================] - 89s 48ms/step - loss: 94.4057 - val_loss: 78.8916

Epoch 00004: val_loss improved from 81.52022 to 78.89157, saving model to ./check\transformer-end_turn_restricted-100-2048.h5
Epoch 5/100
1836/1836 [==============================] - 91s 49ms/

,id,target
0,TEST_000000,24.791613
1,TEST_000001,45.600868
2,TEST_000002,56.051453
3,TEST_000003,28.270933
4,TEST_000004,39.804962
...,...,...
291236,TEST_291236,47.185257
291237,TEST_291237,51.513466
291238,TEST_291238,21.670900
291239,TEST_291239,27.392941
